In [1]:
# Basic Modules
import numpy as np
import pandas as pd
from scipy.cluster.vq import *
import operator
import matplotlib
reload(matplotlib)
matplotlib.use('Agg')
from matplotlib import pyplot as plt   
import pickle as pkl
import shelve
import re
from collections import Counter, defaultdict, OrderedDict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics#v_measure_score
import scipy
from sklearn.feature_extraction import DictVectorizer
from matplotlib.backends.backend_pdf import PdfPages
import csv
import sys
import math
from copy import deepcopy
import random
from datetime import datetime, timedelta
import pickle
import sys
from multiprocessing import Pool
import pp
import shelve
from collections import OrderedDict

# ML modules
from sklearn.ensemble import AdaBoostClassifier
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import OneClassSVM
from sklearn.mixture import GMM
from sklearn.mixture import DPGMM
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB

# Custom modules
from bd_wrapper import BDWrapper

In [2]:
sensor_dict = shelve.open('metadata/bacnet_devices.shelve','r')

In [3]:
buildingName = 'ebu3b'
naeDict = dict()
naeDict['bonner'] = ["607", "608", "609", "557", "610"]
naeDict['ap_m'] = ['514', '513','604']
naeDict['bsb'] = ['519', '568', '567', '566', '564', '565']
naeDict['ebu3b'] = ["505", "506"]
naeList = naeDict[buildingName]
missingSrcidList = list()
with open('missingsrcidlist.csv', 'rb') as fp:
    reader = csv.reader(fp)
    for row in reader:
        missingSrcidList.append(row[0])
outputFilename = 'data/fe_' + buildingName + '.pkl'

In [4]:
def extract_features(srcidList, dummy):
    import feature_extractor as fe
    import shelve
    resultList = list()
    for srcid in srcidList:
        #print srcid
        filename = 'rawdata/ebu3b/'+srcid+'.shelve'
        #with open('rawdata/ebu3b/'+srcid+'.pkl', 'rb') as fp:
            #ts = pickle.load(fp)
        ts = shelve.open(filename)['data']
        #print "hello"
        
        resultList.append((srcid, fe.get_features(ts)))
        #resultList.append((srcid, 10))
        ts = None
    return resultList

In [5]:
def temp_func(srcid):
    return (srcid, 10)

In [6]:
uuidList = list()
srcidList = list()
for nae in naeList:
    deviceList = sensor_dict[nae]
    for sensor in deviceList['objs'][1:]:
        h_obj = sensor['props']
        srcid = nae + '_' + str(h_obj['type']) + '_' + str(h_obj['instance'])
        if not srcid in missingSrcidList:
            srcidList.append(srcid)

In [7]:
print len(srcidList)

4573


In [8]:
#ncpus = 4
#rangeList = list()

#sensorsNum = len(srcidList)
#for i in range(0,ncpus):
#    rangeList.append(range(sensorsNum/ncpus*(i+1) - sensorsNum/ncpus, sensorsNum/ncpus*(i+1)))

#srcidListList = list()
#for oneRange in rangeList:
#oneList =  [srcidList[i] for i in oneRange]
#    srcidListList.append(oneList)
    
#p = Pool(ncpus)
#tempDict = dict((p.map(extract_features, srcidListList)))

In [9]:
#p = Pool(4)
#tempDict = dict((p.map(extract_features, srcidList)))

ppservers = ()
ncpus = 4
rangeList = list()

#srcidList = ['505_0_3000043', '506_0_3000026', '505_0_3000003', '506_0_3000023',  '506_0_3000027']

sensorsNum = len(srcidList)
for i in range(0,ncpus):
    rangeList.append(range(sensorsNum/ncpus*(i+1) - sensorsNum/ncpus, sensorsNum/ncpus*(i+1)))
print "=============="

jobServer = pp.Server(ncpus, ppservers=ppservers)
jobList = list()
for oneRange in rangeList:
    #print [srcidList[i] for i in oneRange]
    jobList.append(jobServer.submit(extract_features, ([srcidList[i] for i in oneRange], True)))

resultList = list()
resultList = [0,0,0,0]
for i, job in enumerate(jobList):
    resultList[i] = job()
#r1 = jobList[0]()
#r2 = jobList[1]()
jobServer.wait()
print "=-============"
#print r1
#print r2

dictList = list()
#print resultList
for result in resultList:
#    print "result: ", result
    dictList = dictList + result
resultDict = dict(dictList)
#print resultDict

#job_server = pp.Server(ncpus, ppservers=ppservers)
#j1 = job_server.submit(extract_features, srcidList[0:1500])

=-============


In [10]:
with open(outputFilename, 'wb') as fp:
    pickle.dump(resultDict, fp)

In [11]:
## TEST FE
import feature_extractor 
reload(feature_extractor)
import feature_extractor as fe
import FATS


srcidDict = OrderedDict()
srcidDict['ZT'] = '506_0_3000485' # 2150 ZT
srcidDict['OCM'] = '506_1_3000497' # 2150 occ clg max
srcidDict['CS'] = '506_1_3000493' # 2150 TS
srcidDict['DC'] = '506_1_3000494' # 2150 

fat = FATS.FeatureSpace(Data=['magnitude', 'time'], featureList=[#'Mean', 'Amplitude', 'Skew', 
                                                                 #'Meanvariance', 
                                                                 #'PeriodLS' 
                                                                 #'Std', 
                                                                 #'MaxSlope', 
                                                                 'SmallKurtosis'
                                                                ])
#for pointType, srcid in srcidDict.iteritems():
#    filename = 'rawdata/ebu3b/'+srcid+'.shelve'
#    ts = shelve.open(filename)['data']
#    #print pointType, fe.get_fft(ts)
#    ts = np.asarray([ts.values, ts.index.values])
#    result = fat.calculateFeature(ts).result()
#    print pointType, result

result = extract_features(srcidDict.values(), True)
print result


[('506_0_3000485', [[78.090000000000003], [70.310000000000002], [73.23721682205932], [7.7800000000000011], [1.3343221657969844], [1], [0.09193601197209314], [0.08667059921117616], [0.2309582309582308], [0.4558330478773272], [41.127274720277448], [10.167352010178156]]), ('506_1_3000497', [[330.0], [330.0], [330.0], [0.0], [0.0], [7], [0.0], [0.0], -999999, -999999, [41.127274720277448], [0.0]]), ('506_1_3000493', [[70.0], [68.0], [69.51229914699465], [2.0], [0.4257477529643602], [0], [0.03649067915102501], [0.0354748255850961], [0.0], [0.2815235026816336], [41.127274720277448], [1.4747474747510783]]), ('506_1_3000494', [[3.1400000000000001], [-2.5499999999999998], [-0.047014481253718446], [5.6899999999999995], [0.2068900312037349], [3], [0.07643021840557931], [0.08918966463222437], [0.0], [0.08265730251103016], [41.127274720277448], [16.398902542401341]])]
